# Proyecto Final DS Parte III - Hamburg

Este notebook complementa el análisis exploratorio anterior incorporando técnicas de **selección de características**, un modelo supervisado de **clasificación**, evaluación con **métricas básicas** y conclusiones.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Cargar dataset desde GitHub (ajustar ruta según tu repo)
url = 'https://raw.githubusercontent.com/gerardohamburg/CoderHouse/main/songs_normalize.csv'
df = pd.read_csv(url)
df.head()

## Selección de características: PCA
Reducimos la dimensionalidad manteniendo al menos el 90% de la varianza.

In [ ]:
X = df.select_dtypes(include=[np.number]).dropna(axis=1)
y = (df['popularity'] >= df['popularity'].median()).astype(int)

pca = PCA(n_components=0.90)
X_pca = pca.fit_transform(X)
print('Forma original:', X.shape)
print('Forma reducida:', X_pca.shape)

## Entrenamiento y Validación
Se utiliza una **Regresión Logística** como modelo base.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, stratify=y, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.show()

## Conclusiones
- PCA permitió reducir significativamente el número de variables manteniendo más del 90% de la varianza.
- La regresión logística es un buen modelo de línea base para clasificar popularidad.
- Las métricas obtenidas muestran un balance aceptable entre precisión y recall.
- Este pipeline es completamente reproducible en Colab y usa únicamente herramientas vistas en el curso.